In [1]:
import json
import pandas as pd
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact
from ipywidgets import FloatSlider
from scipy.spatial import procrustes
from ipywidgets import Play

In [2]:
with open('../jupyters/new_circle_positions.pkl', 'rb') as f:
    new_circle_positions = pickle.load(f)

#### Traditional way

In [3]:
def plot_circles(lambda_):
    data = new_circle_positions
    new_circle_positions_selected = data[lambda_]
    x = [i[0] for i in new_circle_positions_selected]
    y = [i[1] for i in new_circle_positions_selected]
    plt.figure()
    plt.plot(x, y, 'o'); # markersize = 15

    n = range(len(x))
    for i, txt in enumerate(n):
        plt.annotate(txt, (x[i], y[i]), fontsize =15)

    plt.show()


In [4]:
interact(plot_circles, lambda_=FloatSlider(min=0, max=1, step=0.01));

interactive(children=(FloatSlider(value=0.0, description='lambda_', max=1.0, step=0.01), Output()), _dom_class…

#### Apply procrustes

In [5]:
#get new matrix according to procrustes

lambdas = list(new_circle_positions.keys())
standardized_matrix = dict()
disparity_values = dict()
original_a = new_circle_positions[0.0]
for i in range(len(lambdas)-1):
    #print(lambdas[i], lambdas[i+1])
    original_b = new_circle_positions[lambdas[i+1]]
    mtx1, mtx2, disparity = procrustes(original_a, original_b)
    disparity_values[lambdas[i]] = disparity
    standardized_matrix[lambdas[i]] = mtx1.tolist()
    original_a = mtx2
    
    
standardized_matrix[lambdas[len(lambdas)-1]] = mtx2
disparity_values[lambdas[len(lambdas)-1]] = disparity
    

with open('../jupyters/standardized_matrix.pkl', 'wb') as f:
    pickle.dump(standardized_matrix, f)

In [6]:
def plot_circles_with_procrustes(lambda_):
    data = standardized_matrix
    new_circle_positions_selected = data[lambda_]
    #print(lambda_, new_circle_positions_selected)
    x = [i[0] for i in new_circle_positions_selected]
    y = [i[1] for i in new_circle_positions_selected]
    plt.figure()
    plt.plot(x, y, 'o'); # markersize = 15

    n = range(len(x))
    for i, txt in enumerate(n):
        plt.annotate(txt, (x[i], y[i]), fontsize =15)

    plt.show()


In [7]:
interact(plot_circles_with_procrustes, lambda_=FloatSlider(min=0, max=1, step=0.01));

interactive(children=(FloatSlider(value=0.0, description='lambda_', max=1.0, step=0.01), Output()), _dom_class…

### Compare| traditional ways vs procrustes

In [8]:
def compare_traditional_vs_procrustes(lambda_):
    lambda_ = lambda_/100.0

    data = new_circle_positions
    new_circle_positions_selected = data[lambda_]
    x_traditional = [i[0] for i in new_circle_positions_selected]
    y_traditional = [i[1] for i in new_circle_positions_selected]
    
    #procrustes data
    data = standardized_matrix
    new_circle_positions_selected = data[lambda_]
    x_procrustes = [i[0] for i in new_circle_positions_selected]
    y_procrustes = [i[1] for i in new_circle_positions_selected]
    
    #plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
    fig.suptitle('Similarity between topics with omega = '+str(lambda_)+" disparity ="+str(np.format_float_scientific(disparity_values[lambda_], precision=2)), fontsize=16)
    ax1.plot(x_traditional, y_traditional,'o')
    ax1.title.set_text('Traditional way')
    ax2.plot(x_procrustes, y_procrustes,'o')
    ax2.title.set_text('With Procrustes')
    
    
    
    n = range(len(x_traditional))
    for i, txt in enumerate(n):
        ax1.annotate(txt, (x_traditional[i], y_traditional[i]), fontsize =15)
        ax2.annotate(txt, (x_procrustes[i], y_procrustes[i]), fontsize =15)
    
    


In [9]:
#s = disparity_values[0]
#np.format_float_scientific(s,  precision=2, exp_digits=1)

In [10]:
#compare_traditional_vs_procrustes(0.0)
#interact(compare_traditional_vs_procrustes, lambda_=FloatSlider(min=0, max=1, step=0.01));
interact(compare_traditional_vs_procrustes, lambda_=Play(value=0, min=0, max=100, step=1, interval=200));


interactive(children=(Play(value=0, description='lambda_', interval=200), Output()), _dom_classes=('widget-int…